In [ ]:
#r "nuget: BserClient, 1.1.1"

Installed Packages BserClient, 1.1.1

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.IO;
using BserClient;
using System.Collections.Generic;

string readText = File.ReadAllText("games.json");

List<BserClient.Types.UserGameObj> games = 
JsonSerializer.Deserialize<List<BserClient.Types.UserGameObj>>(readText);

// List<dynamic> games = JsonSerializer.Deserialize<List<dynamic>>(readText);
// var games = Json.Decode(readText);
int ohioGames = 0;
int koreaGames = 0;
foreach (var game in games)
{
    if (game.matchingMode == 3 && game.seasonId == 3) {
        if (game.serverName == "Ohio") ohioGames += 1;
        if (game.serverName == "Seoul")  koreaGames += 1;
    }
}

// get ohioGames vs korea games
Console.WriteLine(ohioGames);
Console.WriteLine(koreaGames);

480


331


In [ ]:
foreach (var game in games.GroupBy(x => x.characterNum))
{
    Console.WriteLine("Events for year: " + game.Key);
    foreach (var e in game) 
    {
        Console.WriteLine(e);
    }
    // eventsInYear.Key - year
    // eventsInYear - collection of events in that year
}